# GAN para xerar díxitos manuscritos (MNIST)

Neste caderno, construiremos e adestraremos unha Rede Antagónica Xerativa (GAN, do inglés *Generative Adversarial Network*) para crear imaxes de díxitos manuscritos convincentes.

## Como funciona unha GAN?

Unha GAN consta de dúas redes neuronais que compiten entre si:

1.  **O Xerador (Generator)**: Tenta crear imaxes falsas que sexan o máis realistas posible. Comeza a partir de ruído aleatorio e aprende a transformalo en algo que se pareza aos datos reais.
2.  **O Discriminador (Discriminator)**: Actúa como un xuíz. O seu traballo é observar unha imaxe e decidir se é real (do conxunto de datos de adestramento) ou falsa (creada polo xerador).

O adestramento é un xogo de suma cero:
- O **discriminador** mellora ao detectar mellor as falsificacións.
- O **xerador** mellora ao "enganar" mellor ao discriminador.

Co tempo, o xerador vólvese tan bo que as súas imaxes son case indistinguibles das reais.

## 1. Configuración do Entorno

Primeiro, importamos todas as librarías que necesitaremos.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import os

print(f"Versión de TensorFlow: {tf.__version__}")

## 2. Carga e Preparación dos Datos

Usaremos o clásico dataset MNIST. As imaxes normalizaranse para que os valores dos píxeles estean no rango de [-1, 1], o que axuda a estabilizar o adestramento da GAN.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Cargamos o conxunto de adestramento de MNIST
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

print("\nMostrando as primeiras 100 imaxes de exemplo:")

# 1. Axustamos o tamaño da figura para que entren 100 imaxes comodamente
plt.figure(figsize=(5, 5)) 

# 2. Cambiamos o bucle para que se repita 100 veces
for i in range(100):
    # 3. Creamos unha reixa (grid) de 10x10 para os subplots
    plt.subplot(10, 10, i + 1)
    
    # Mostramos a imaxe. 'cmap=plt.cm.binary' amóstraas en branco e negro.
    # CORRECCIÓN: Usamos train_images e train_labels (as variables que cargamos)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    
    plt.xticks([]) # Quitamos as marcas do eixe X
    plt.yticks([]) # Quitamos as marcas do eixe Y
    

plt.suptitle("100 Imaxes de Exemplo do Dataset MNIST (Train Set)", fontsize=20)
plt.show() # Mostramos a figura con todas as imaxes

# --- O TEU CÓDIGO DE PREPROCESADO (non necesita cambios) ---

# Para non sobrescribir as imaxes orixinais que usamos para visualizar,
# creo unha nova variable para as imaxes procesadas.
train_images_processed = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

# Normalizar as imaxes ao rango [-1, 1]
train_images_processed = (train_images_processed - 127.5) / 127.5

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Crear lotes de datos e barallar
train_dataset = tf.data.Dataset.from_tensor_slices(train_images_processed).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print("\nDataset listo para o adestramento.")

## 3. Creación do Modelo Xerador

O xerador toma un vector de ruído aleatorio (de dimensión 100) como entrada e produce unha imaxe de 28x28x1.

Usamos capas `Conv2DTranspose` para facer "upsampling", é dicir, para pasar dun vector pequeno a unha imaxe completa.

In [ ]:
def make_generator_model():
    model = models.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # None é o tamaño do batch

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # A capa final usa 'tanh' porque normalizamos as nosas imaxes a [-1, 1]
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

generator = make_generator_model()
generator.summary()

## 4. Creación do Modelo Discriminador

O discriminador é un clasificador de imaxes convencional (unha CNN). Toma unha imaxe de 28x28x1 como entrada e devolve unha única probabilidade: a probabilidade de que a imaxe sexa real.

In [5]:
def make_discriminator_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    # Non engadimos 'sigmoid' aquí porque a función de perda o fará por nós.

    return model

discriminator = make_discriminator_model()
discriminator.summary()

/home/fablabs/jupyterhub/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Definición das Perdas e Optimizadores

- **Función de Perda**: Usamos `BinaryCrossentropy` para ambas redes. O discriminador quere minimizar esta perda (clasificar correctamente), mentres que o xerador quere maximizala (enganar ao discriminador).
- **Optimizadores**: `Adam` é unha boa elección estándar para as GANs.

In [6]:
# Esta función de perda calcula a entropía cruzada entre as etiquetas verdadeiras e as predicións.
# O argumento `from_logits=True` indica que a saída do modelo non está normalizada (non pasou por sigmoid).
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    # A perda para as imaxes reais: compara as predicións do discriminador sobre imaxes reais con etiquetas de "1" (real).
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # A perda para as imaxes falsas: compara as predicións do discriminador sobre imaxes falsas con etiquetas de "0" (falso).
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # A perda do xerador: se o xerador funciona ben, o discriminador clasificará as súas imaxes como reais (etiqueta 1).
    # Por tanto, comparamos as decisións do discriminador sobre as imaxes falsas con etiquetas de "1".
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = optimizers.Adam(1e-4)
discriminator_optimizer = optimizers.Adam(1e-4)

## 6. Configuración do Bucle de Adestramento

Este é o paso máis complexo. Para cada época de adestramento:

1.  Xeramos un lote de imaxes falsas a partir de ruído.
2.  Collemos un lote de imaxes reais do dataset.
3.  **Adestramos o Discriminador**: Ensinámoslle a distinguir as imaxes reais das falsas.
4.  **Adestramos o Xerador**: Ensinámoslle a crear imaxes que o discriminador clasifique como reais.

Tamén gardaremos puntos de control (checkpoints) e xeraremos imaxes de mostra periodicamente para visualizar o progreso.

In [ ]:
EPOCHS = 30 # Podes comezar con menos (ex. 50) para un adestramento máis rápido
noise_dim = 100
num_examples_to_generate = 16

# Usaremos o mesmo ruído de entrada cada vez para visualizar mellor o progreso do xerador
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# O decorador `tf.function` compila a función nun grafo de TensorFlow para un rendemento máis rápido.
@tf.function
def train_step(images):
    # Xerar ruído para o xerador
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    # Usamos `tf.GradientTape` para rexistrar as operacións e calcular os gradientes.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calcular os gradientes e aplicalos aos modelos
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Crear un directorio para gardar as imaxes xeradas
if not os.path.exists('gan_images'):
    os.makedirs('gan_images')

def generate_and_save_images(model, epoch, test_input):
    # A predición faise en modo non-adestramento (`training=False`)
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Desnormalizar a imaxe de [-1, 1] a [0, 1] para visualizala
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('gan_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

## 7. Adestramento da GAN

Agora, poñemos todo xunto e comezamos o adestramento. Este proceso pode tardar bastante tempo, dependendo da túa GPU.

In [ ]:
from IPython import display
import time

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Xerar imaxes de mostra ao final de cada época
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        print ('Tempo para a época {} é de {} seg'.format(epoch + 1, time.time()-start))

    # Xerar imaxes finais
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)

In [ ]:
# Comezar o adestramento!
train(train_dataset, EPOCHS)

## 8. Conclusión

Parabéns! Adestraches unha GAN dende cero.

Podes ollar o cartafol `gan_images` para ver a evolución do xerador. Ao principio, as imaxes serán puro ruído, pero co paso das épocas, deberían empezar a tomar a forma de díxitos recoñecibles.

### Posibles seguintes pasos:
- **Adestrar máis tempo**: As GANs benefícianse de máis épocas de adestramento.
- **Axustar hiperparámetros**: Proba con diferentes taxas de aprendizaxe (`learning rates`), tamaños de batch, ou arquitecturas de rede.
- **Probar con outros datasets**: Podes adaptar esta GAN para adestrala con outros conxuntos de datos, como [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) ou [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).